Dungeon AI V1

In [1]:
# module installation script
%pip install google-generativeai

   ---------------------------------------- 0.0/155.4 kB ? eta -:--:--
   ---------------------------------------- 155.4/155.4 kB 4.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   -------------- ------------------------- 0.5/1.3 MB 10.2 MB/s eta 0:00:01
   -------------------------- ------------- 0.9/1.3 MB 9.4 MB/s eta 0:00:01
   ---------------------------------------  1.3/1.3 MB 10.5 MB/s eta 0:00:01
   ---------------------------------------- 1.3/1.3 MB 9.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/160.1 kB ? eta -:--:--
   ---------------------------------------- 160.1/160.1 kB 9.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/216.0 kB ? eta -:--:--
   --------------------------------------- 216.0/216.0 kB 13.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/13.3 MB ? eta -:--:--
    --------------------------------------- 0.3/13.3 MB 6.5 MB/s eta 0:00:02
   -- -------------------


[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [16]:
# --- IMPORTS ---
import os
import time
import google.generativeai as genai
from dotenv import load_dotenv
import ipywidgets as widgets
from IPython.display import display, clear_output, Markdown

# --- LOAD API KEY ---
load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY")
if not api_key:
    raise ValueError("❌ GOOGLE_API_KEY not found in .env file!")

genai.configure(api_key=api_key)

# --- GAME STATE ---
model_name = "gemini-2.0-flash"
player_name = "Ihno"
context = ""
game_memory = []
player_stats = {}
inventory = []
difficulty = 1

# --- DIFFICULTY ADJUSTMENT ---
def adjust_difficulty(player_input):
    global difficulty
    if any(word in player_input.lower() for word in ["attack", "fight", "battle"]):
        difficulty = min(difficulty + 1, 5)
    elif any(word in player_input.lower() for word in ["run", "talk", "hide"]):
        difficulty = max(difficulty - 1, 1)
    return difficulty

# --- STORY GENERATION (Gemini) ---
def generate_story(context, player_input, difficulty):
    prompt = (
        "You are a fantasy dungeon-master AI. "
        "Continue the adventure in a vivid, immersive style, but keep your response concise (max 5 sentences). "
        "Do not repeat the player's action or question, only narrate what happens next. "
        "Try to keep the story interactive and engaging. "
        f"Difficulty level: {difficulty}.\n\n"
        f"{context}\n"
        "Narrator:"
    )

    try:
        model = genai.GenerativeModel(model_name)
        response = model.generate_content(prompt)
        return response.text.strip()
    except Exception as e:
        return f"❌ Error generating story: {e}"

# --- PRINT GAME STATE ---
def print_game_state():
    display(Markdown(f"### 📖 **Story so far**\n{context}"))
    display(Markdown(f"**🧍 {player_name}'s Inventory:** {inventory}"))
    display(Markdown(f"**❤️ Stats:** {player_stats} | 🎯 Difficulty:** {['Easy', 'Medium', 'Hard', 'Very Hard', 'Nightmare'][difficulty - 1]}"))

# --- PLAY TURN ---
def play_turn(player_input):
    global context
    if not player_input.strip():
        return
    game_memory.append(f"{player_name}: {player_input}")
    adjust_difficulty(player_input)

    recent_context = "\n".join(game_memory[-6:])
    new_story = generate_story(recent_context, player_input, difficulty)
    context_update = f"\n{player_name}: {player_input}\n{new_story}"
    context += context_update
    game_memory.append(new_story)

    output_area.clear_output(wait=True)
    with output_area:
        print_game_state()

# --- START GAME ---
def start_new_game():
    global context, player_stats, inventory, difficulty, game_memory
    context = f"{player_name} awakens in a dark forest. A mysterious figure approaches."
    game_memory = [context]
    player_stats = {"health": 100, "strength": 10, "gold": 5}
    inventory = ["torch"]
    difficulty = 1
    output_area.clear_output()
    with output_area:
        display(Markdown("🆕 **New game started.**"))
        print_game_state()

# --- UI SETUP ---
input_box = widgets.Text(
    placeholder='What does Ihno do next?',
    description='▶️ Action:',
    layout=widgets.Layout(width='70%')
)
submit_button = widgets.Button(description="Submit", button_style='success')
output_area = widgets.Output()

def on_button_click(b):
    play_turn(input_box.value)
    input_box.value = ''

submit_button.on_click(on_button_click)

# --- START & DISPLAY ---
# if "game_ui_initialized" not in globals():
start_new_game()
display(input_box, submit_button, output_area)
    # game_ui_initialized = True
